## arrcat

makes the HELCATS HI ARRIVAL catalog

Author: C. Möstl, IWF Graz, Austria
twitter @chrisoutofspace, part of https://github.com/cmoestl/heliocats

last update 12 May 2020, current version ARRCAT 2.0

Install a specific conda environment to run this code, see readme at https://github.com/cmoestl/heliocats

Convert this notebook to a script with "jupyter nbconvert --to script arrcat.ipynb", automatically done in first cell


In [1]:
import numpy as np
import scipy.io
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import  DateFormatter
from datetime import timedelta
import seaborn as sns
import datetime
import astropy
import astropy.constants as const
from sunpy.time import parse_time
import time
import pickle
import sys
import os
import urllib
import json
import importlib
import pandas as pd
import copy
import openpyxl
import h5py
import heliopy

from heliocats import plot as hp
importlib.reload(hp) #reload again while debugging

from heliocats import data as hd
importlib.reload(hd) #reload again while debugging

from heliocats import cats as hc
importlib.reload(hc) #reload again while debugging

from heliocats import stats as hs
importlib.reload(hs) #reload again while debugging

#where the in situ data files are located is read 
#from config.py 
import config
importlib.reload(config)
from config import data_path
from config import data_path_ML

########### make directories first time if not there

resdir='results'
if os.path.isdir(resdir) == False: os.mkdir(resdir)

datadir='data'
if os.path.isdir(datadir) == False: os.mkdir(datadir)

catdir='arrcat'
if os.path.isdir(catdir) == False: os.mkdir(catdir)

icplotsdir='arrcat/plots_arrcat/' 
if os.path.isdir(icplotsdir) == False: os.mkdir(icplotsdir) 

#Convert this notebook to a script with jupyter nbconvert --to script icmecat.ipynb
os.system('jupyter nbconvert --to script arrcat.ipynb')    

0

## 1 Make HI SSEF30 arrival catalog ARRCAT

In [2]:
from heliocats import cats as hc
importlib.reload(hc) #reload again while debugging 


#https://www.helcats-fp7.eu/
#LOAD HELCATS HIGeoCAT
url_higeocat='https://www.helcats-fp7.eu/catalogues/data/HCME_WP3_V06.vot'

try: urllib.request.urlretrieve(url_higeocat,'data/HCME_WP3_V06.vot')
except urllib.error.URLError as e:
    print('higeocat not loaded')

higeocat=hc.load_higeocat_vot('data/HCME_WP3_V06.vot')
higeocat_time=parse_time(higeocat['Date']).datetime    
higeocat_t0=parse_time(higeocat['SSE Launch']).datetime   #backprojected launch time

#define empty pandas dataframe for arrival catalog with column names

column_list=['id', 'sc','target_name','sse_launch_time','target_arrival_time','target_arrival_time_err',\
                                 'target_distance','target_heeq_lon','target_heeq_lat',\
                                 'target_delta','sse_heeq_lon','sse_heeq_lat',\
                                 'sse_speed','sse_speed_err','target_speed','target_speed_err',\
                                 'pa_fit','pa_n','pa_s','pa_center']
             
#pandas dataframe for current version with iteration in calculating arrival time
ac = pd.DataFrame([], columns = column_list)

#pandas dataframe for old method of arrival time prediction without iteration
ac_old = pd.DataFrame([], columns = column_list)

#Make arrival catalog from HIGEOCAT for each spacecraft

[ac,ac_old]=hc.make_arrival_catalog_insitu_ssef30(higeocat, ac, ac_old, 'PSP',column_list)
[ac,ac_old]=hc.make_arrival_catalog_insitu_ssef30(higeocat, ac, ac_old, 'Solo',column_list)
[ac,ac_old]=hc.make_arrival_catalog_insitu_ssef30(higeocat, ac, ac_old, 'Bepi',column_list)
   
[ac,ac_old]=hc.make_arrival_catalog_insitu_ssef30(higeocat, ac, ac_old, 'STA',column_list)
[ac,ac_old]=hc.make_arrival_catalog_insitu_ssef30(higeocat, ac, ac_old, 'STB',column_list)
[ac,ac_old]=hc.make_arrival_catalog_insitu_ssef30(higeocat, ac, ac_old, 'Earth',column_list)

[ac,ac_old]=hc.make_arrival_catalog_insitu_ssef30(higeocat, ac, ac_old, 'Ulysses',column_list)
[ac,ac_old]=hc.make_arrival_catalog_insitu_ssef30(higeocat, ac, ac_old, 'Mercury',column_list)
[ac,ac_old]=hc.make_arrival_catalog_insitu_ssef30(higeocat, ac, ac_old, 'Venus',column_list)
[ac,ac_old]=hc.make_arrival_catalog_insitu_ssef30(higeocat, ac, ac_old, 'Mars',column_list)


ac = ac.reset_index(drop=True)
ac_old = ac_old.reset_index(drop=True)


ac

SSEF30 events:  5
PSP  SSEF30 arrival catalog finished.

SSEF30 events:  4
Solo  SSEF30 arrival catalog finished.

SSEF30 events:  18
Bepi  SSEF30 arrival catalog finished.

SSEF30 events:  141
STA  SSEF30 arrival catalog finished.

SSEF30 events:  139
STB  SSEF30 arrival catalog finished.

SSEF30 events:  530
Earth  SSEF30 arrival catalog finished.

SSEF30 events:  22
Ulysses  SSEF30 arrival catalog finished.

SSEF30 events:  209
Mercury  SSEF30 arrival catalog finished.

SSEF30 events:  250
Venus  SSEF30 arrival catalog finished.

SSEF30 events:  230
Mars  SSEF30 arrival catalog finished.



,id,sc,target_name,sse_launch_time,target_arrival_time,target_arrival_time_err,target_distance,target_heeq_lon,target_heeq_lat,target_delta,sse_heeq_lon,sse_heeq_lat,sse_speed,sse_speed_err,target_speed,target_speed_err,pa_fit,pa_n,pa_s,pa_center
0,HCME_A__20180828_01,A,PSP,2018-08-28 14:13,2018-09-01 20:00,21.8,0.969,-7.92,5.86,-5.9,-2,0,400,82,396,81,270,255,280,267.5
1,HCME_A__20180919_01,A,PSP,2018-09-19 00:21,2018-09-21 11:16,2.4,0.846,-13.51,4.83,-13.5,0,4,633,26,597,25,275,260,290,275.0
2,HCME_A__20181026_01,A,PSP,2018-10-26 04:32,2018-10-27 22:51,2.1,0.330,10.47,1.33,26.5,-16,-19,433,21,324,16,250,210,260,235.0
3,HCME_A__20190402_01,A,PSP,2019-04-02 02:56,2019-04-02 22:39,0.1,0.180,-81.93,-2.98,-12.9,-69,4,400,3,379,3,285,270,290,280.0
4,HCME_A__20190513_01,A,PSP,2019-05-13 16:22,2019-05-15 17:35,9.1,0.796,69.65,3.54,19.6,50,0,768,137,672,120,260,235,280,257.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1543,HCME_A__20180828_01,A,Mars,2018-08-28 14:13,2018-09-03 19:19,31.9,1.383,-12.89,5.08,-10.9,-2,0,400,82,385,79,270,255,280,267.5
1544,HCME_A__20180919_01,A,Mars,2018-09-19 00:21,2018-09-23 08:01,4.3,1.382,-19.62,5.50,-19.6,0,4,633,26,554,23,275,260,290,275.0
1545,HCME_A__20181026_01,A,Mars,2018-10-26 04:32,2018-11-01 09:25,7.2,1.396,-33.68,5.55,-17.7,-16,-19,433,21,390,19,250,210,260,235.0
1546,HCME_A__20181211_01,A,Mars,2018-12-11 02:44,2018-12-17 09:33,0.4,1.437,-51.94,4.40,7.1,-59,8,402,1,396,1,275,260,280,270.0


## 2 save ARRCAT


#### save header

In [3]:
#save header and parameters as text file and prepare for html website
header='ARRIVAL CATALOGUE 2.0 \n\n\
In this ARRival CATalog (ARRCAT), the arrivals of solar coronal mass ejections as \n\
tracked in the STEREO heliospheric imagers are calculated. This was a part of the EU HELCATS project, \n\
but is now continously updated. \n\
It lists predicted arrivals of CMEs at various spacecraft and planets, \n\
between April 2007 - April 2020, based on the HIGeoCAT catalog of CMEs \n\
established at RAL Space, UK (R. A. Harrison, J. A. Davies, D. Barnes). \n\n\
This is version 2.0, released 2020-05-12. DOI: 10.6084/m9.figshare.12271292\n\n\
It is based on this HIGeoCAT version: '+url_higeocat+'\n\n\
The catalog is available as python pandas dataframe (pickle), \n\
python numpy arrays (pickle, as recarray and structured array), \n\
npy (numpy, no pickle), json, csv, xlsx, txt, hdf5, html at \n\
https://helioforecast.space/arrcat and https://www.helcats-fp7.eu\n\n\
Number of events in ARRCAT: '+str(len(ac))+'\n\
Targets: Earth_L1, STEREO-A, STEREO-B, Solar Orbiter, Parker Solar Probe (PSP), Bepi Colombo, Ulysses, Venus, Mercury, Mars.\n\n\
Parameters: \n\n\
    0: id: From HIGeoCAT, the unique identifier for the observed CME.\n\
    1: sc: From HIGeoCAT, the HI observing STEREO spacecraft, (A=Ahead or B=Behind)\n\
    2: target_name: Name of in situ target.\n\
    3: sse_launch_time: From HIGeoCAT, launch time of the CME at Sun center, unit: UTC.\n\
    4: target_arrival_time: CME arrival time at target location, corrected for SSE shape (Möstl and Davies, 2013). unit: UTC.\n\
    5: target_arrival_time_err: Error of CME arrival time at target location, solely based on error in parameter sse_speed_err. unit: UTC.\n\
    6: target_distance: Target distance from Sun, at CME arrival time. unit: AU.\n\
    7: target_heeq_lon: Target longitude in HEEQ, at CME arrival time. unit: degree.\n\
    8: target_heeq_lat: Target latitude in HEEQ, at CME arrival time. unit: degree.\n\
    9: target_delta: Difference in HEEQ longitude between central CME direction and target location at CME arrival time, \n\
       positive values: spacecraft is west of CME apex. unit: degree.\n\
    10: sse_heeq_lon: From HIGeoCAT, the HEEQ longitude of the CME apex propagation direction, unit: degree.\n\
    11: sse_heeq_lat: From HIGeoCAT, the HEEQ latitude of the CME apex propagation direction, unit: degree.\n\
    12: sse_speed: From HIGeoCAT, speed of the CME apex, unit: km/s.\n\
    13: sse_speed_err: From HIGeoCAT, error in the speed of the CME apex, unit: km/s.\n\
    14: target_speed: CME arrival speed at target location, corrected for SSE shape (Möstl and Davies, 2013). unit: km/s.\n\
    15: target_speed_err: Error in CME arrival speed at target location, corrected for SSE shape (Möstl and Davies, 2013). unit: km/s.\n\
    16: pa_fit: From HIGeoCAT, PA along which time-elongation profile is extracted, unit: degree.\n\
    17: pa_n: From HIGeoCAT, northern position angle of CME, unit: degree.\n\
    18: pa_s: From HIGeoCAT, southernmost position angle of CME, unit: degree.\n\
    19: pa_center: average of pa_n and pa_s, unit: degree.\n\n\
Notes:\n\n\
    - We have modified the calculation of CME arrival time here by a new iterative method compared to Möstl et al. (2014, 2017). \n\
      In the first iteration, the sse_launch_time is used to calculate the target HEEQ position, and in subsequent three iterations the \n\
      newly calculated arrival time is employed each time for the target HEEQ position at the newly calculated arrival time. \n\
      In this way we avoid an error of a few degrees in the arrival target location (see also Möstl et al. 2017). \n\
    - The analytical formulas for calculating the speeds and arrival times of the CMEs modeled with SSEF30, \n\
      corrected for the SSEF30 circular shape, can be found in Möstl & Davies (2013). \n\n\n\
References \n\n\
Möstl & Davies (2013) https://doi.org/10.1007/s11207-012-9978-8 \n\
Möstl et al. (2014)   https://doi.org/10.1088/0004-637X/787/2/119 \n\
Möstl et al. (2017)   https://doi.org/10.1002/2017SW001614 \n'

print(header)

#make header file
file='arrcat/HELCATS_ARRCAT_v20_header.txt'
with open(file, "w") as text_file:
    text_file.write(header)
print()    
print('header saved as '+file)
print()    

#Convert to html regarding line breaks, paragraph beginning and spaces
header_spaces=header.replace(" ", "&nbsp;")
header_html= "<p>" +header_spaces.replace('\n', '<br>')+ "</p>" 
print('header converted to HTML')
print()    
print()    

ARRIVAL CATALOGUE 2.0 

In this ARRival CATalog (ARRCAT), the arrivals of solar coronal mass ejections as 
tracked in the STEREO heliospheric imagers are calculated. This was a part of the EU HELCATS project, 
but is now continously updated. 
It lists predicted arrivals of CMEs at various spacecraft and planets, 
between April 2007 - April 2020, based on the HIGeoCAT catalog of CMEs 
established at RAL Space, UK (R. A. Harrison, J. A. Davies, D. Barnes). 

This is version 2.0, released 2020-05-12. DOI: 10.6084/m9.figshare.12271292

It is based on this HIGeoCAT version: https://www.helcats-fp7.eu/catalogues/data/HCME_WP3_V06.vot

The catalog is available as python pandas dataframe (pickle), 
python numpy arrays (pickle, as recarray and structured array), 
npy (numpy, no pickle), json, csv, xlsx, txt, hdf5, html at 
https://helioforecast.space/arrcat and https://www.helcats-fp7.eu

Number of events in ARRCAT: 1548
Targets: Earth_L1, STEREO-A, STEREO-B, Solar Orbiter, Parker Solar Probe (

#### save into different formats

In [4]:
########## python formats

# save ARRCAT as pandas dataframe with times as datetime objects as pickle
file='arrcat/HELCATS_ARRCAT_v20_pandas.p'
pickle.dump([ac,header], open(file, 'wb'))
print('ARRCAT saved as '+file)

# save ARRCAT as numpy array with times as matplotlib datetime as pickle
ac_num=copy.deepcopy(ac) 
ac_num.sse_launch_time=parse_time(ac_num.sse_launch_time).plot_date
ac_num.target_arrival_time=parse_time(ac_num.target_arrival_time).plot_date
#convert to recarray
ac_num_rec=ac_num.to_records()
#create structured array 
dtype1=[('index','i8'),('id', '<U30'),('sc', '<U20'),('target_name', '<U20')] +[(i, '<f8') for i in ac.keys()[3:len(ac.keys())]]
ac_num_struct=np.array(ac_num_rec,dtype=dtype1)

file='arrcat/HELCATS_ARRCAT_v20_numpy.p'
pickle.dump([ac_num,ac_num_struct,header], open(file, 'wb'))
print('ARRCAT saved as '+file)




################ save to different formats

#copy pandas dataframe first to change time format consistent with HELCATS
ac_copy=copy.deepcopy(ac)  
#change time format and add Z
ac_copy.sse_launch_time=parse_time(ac.sse_launch_time).isot
ac_copy.target_arrival_time=parse_time(ac.target_arrival_time).isot

for i in np.arange(len(ac)):

    dum=ac_copy.sse_launch_time[i] 
    ac_copy.at[i,'sse_launch_time']=dum[0:16]+'Z'
     
    dum=ac_copy.target_arrival_time[i] 
    ac_copy.at[i,'target_arrival_time']=dum[0:16]+'Z'


#save as Excel
file='arrcat/HELCATS_ARRCAT_v20.xlsx'
ac_copy.to_excel(file,sheet_name='ARRCATv2.0')
print('ARRCAT saved as '+file)

#save as json
file='arrcat/HELCATS_ARRCAT_v20.json'
ac_copy.to_json(file)
print('ARRCAT saved as '+file)

#save as csv
file='arrcat/HELCATS_ARRCAT_v20.csv'
ac_copy.to_csv(file)
print('ARRCAT saved as '+file)

#save as txt
file='arrcat/HELCATS_ARRCAT_v20.txt'
np.savetxt(file, ac_copy.values.astype(str), fmt='%s' )
print('ARRCAT saved as '+file)



#**************************########### save old version for comparison

#copy pandas dataframe first to change time format consistent with HELCATS
ac_copy_old=copy.deepcopy(ac_old)  
#change time format and add Z
ac_copy_old.sse_launch_time=parse_time(ac_old.sse_launch_time).isot
ac_copy_old.target_arrival_time=parse_time(ac_old.target_arrival_time).isot

for i in np.arange(len(ac_old)):

    dum=ac_copy_old.sse_launch_time[i] 
    ac_copy_old.at[i,'sse_launch_time']=dum[0:16]+'Z'
     
    dum=ac_copy_old.target_arrival_time[i] 
    ac_copy_old.at[i,'target_arrival_time']=dum[0:16]+'Z'


#save as Excel
file='arrcat/HELCATS_ARRCAT_v10_old.xlsx'
ac_old.to_excel(file,sheet_name='ARRCATv2.0')
print('ARRCAT saved as '+file)





#########################

#########save into hdf5 format , use S for strings http://docs.h5py.org/en/stable/strings.html#what-about-numpy-s-u-type
dtype2=[('index','i8'),('id', 'S30'),('sc', 'S20'),('target_name', 'S20')] +[(i, '<f8') for i in ac.keys()[3:len(ac.keys())]]
ach5=np.array(ac_num_rec,dtype=dtype2)
file='arrcat/HELCATS_ARRCAT_v20.h5'
f=h5py.File(file,mode='w')
f["arrcat"]= ach5

#add attributes
#************************
#***********************

print('ARRCAT saved as '+file)
f.close()

#reading h5py files http://docs.h5py.org/en/latest/quick.html
#fr = h5py.File('icmecat/HELCATS_ICMECAT_v20.h5', 'r')
#list(fr.keys())
#ich5=fr['icmecat']
#ich5['mo_bstd']
#ich5.dtype
#fr.close()
##################


#save as .npy without pickle
file='arrcat/HELCATS_ARRCAT_v20_numpy.npy'
np.save(file,ach5, allow_pickle=False)
print('ARRCAT saved as '+file)

#for loading do:
#icnpy=np.load(file)
#decode strings:
#icnpy['icmecat_id'][0].decode()



############ other formats

#copy pandas dataframe first to change time format 
ac_copy2=copy.deepcopy(ac)  
ac_copy2.sse_launch_time=parse_time(ac.sse_launch_time).iso
ac_copy2.target_arrival_time=parse_time(ac.target_arrival_time).iso

#change time format
for i in np.arange(len(ac)):

    dum=ac_copy2.sse_launch_time[i] 
    ac_copy2.at[i,'sse_launch_time']=dum[0:16]
     
    dum=ac_copy2.target_arrival_time[i] 
    ac_copy2.at[i,'target_arrival_time']=dum[0:16]


#save as json for webpage with different time format
file='arrcat/HELCATS_ARRCAT_v20_iso.json'
ac_copy2.to_json(file)
print('ARRCAT saved as '+file)

#save as html no header
file='arrcat/HELCATS_ARRCAT_v20_simple.html'
ac_copy.to_html(file)
print('ARRCAT saved as '+file)

############ save as html file with header
#save as html
file='arrcat/HELCATS_ARRCAT_v20.html'
#ic.to_html(file,justify='center')

#ichtml='{% extends "_base.html" %} \n \n {% block content %} \n \n \n '
achtml = header_html
achtml += ac_copy.to_html()
#ichtml +='\n \n {% endblock %}'


with open(file,'w') as f:
    f.write(achtml)
    f.close()
    
print('ARRCAT saved as '+file)    

ARRCAT saved as arrcat/HELCATS_ARRCAT_v20_pandas.p
ARRCAT saved as arrcat/HELCATS_ARRCAT_v20_numpy.p
ARRCAT saved as arrcat/HELCATS_ARRCAT_v20.xlsx
ARRCAT saved as arrcat/HELCATS_ARRCAT_v20.json
ARRCAT saved as arrcat/HELCATS_ARRCAT_v20.csv
ARRCAT saved as arrcat/HELCATS_ARRCAT_v20.txt
ARRCAT saved as arrcat/HELCATS_ARRCAT_v10_old.xlsx
ARRCAT saved as arrcat/HELCATS_ARRCAT_v20.h5
ARRCAT saved as arrcat/HELCATS_ARRCAT_v20_numpy.npy
ARRCAT saved as arrcat/HELCATS_ARRCAT_v20_iso.json
ARRCAT saved as arrcat/HELCATS_ARRCAT_v20_simple.html
ARRCAT saved as arrcat/HELCATS_ARRCAT_v20.html


## 3 load ARRCAT examples

In [5]:
#load icmecat as pandas dataframe
file='arrcat/HELCATS_ARRCAT_v20_pandas.p'
[ac_pandas,h]=pickle.load( open(file, 'rb'))   

#load icmecat as numpy array
file='arrcat/HELCATS_ARRCAT_v20_numpy.p'
[ac_rec,ac_struct,h]=pickle.load( open(file, 'rb'))   


In [6]:
ac_pandas
ac_pandas.keys()

Index(['id', 'sc', 'target_name', 'sse_launch_time', 'target_arrival_time',
       'target_arrival_time_err', 'target_distance', 'target_heeq_lon',
       'target_heeq_lat', 'target_delta', 'sse_heeq_lon', 'sse_heeq_lat',
       'sse_speed', 'sse_speed_err', 'target_speed', 'target_speed_err',
       'pa_fit', 'pa_n', 'pa_s', 'pa_center'],
      dtype='object')

In [7]:
ac

,id,sc,target_name,sse_launch_time,target_arrival_time,target_arrival_time_err,target_distance,target_heeq_lon,target_heeq_lat,target_delta,sse_heeq_lon,sse_heeq_lat,sse_speed,sse_speed_err,target_speed,target_speed_err,pa_fit,pa_n,pa_s,pa_center
0,HCME_A__20180828_01,A,PSP,2018-08-28 14:13,2018-09-01 20:00,21.8,0.969,-7.92,5.86,-5.9,-2,0,400,82,396,81,270,255,280,267.5
1,HCME_A__20180919_01,A,PSP,2018-09-19 00:21,2018-09-21 11:16,2.4,0.846,-13.51,4.83,-13.5,0,4,633,26,597,25,275,260,290,275.0
2,HCME_A__20181026_01,A,PSP,2018-10-26 04:32,2018-10-27 22:51,2.1,0.330,10.47,1.33,26.5,-16,-19,433,21,324,16,250,210,260,235.0
3,HCME_A__20190402_01,A,PSP,2019-04-02 02:56,2019-04-02 22:39,0.1,0.180,-81.93,-2.98,-12.9,-69,4,400,3,379,3,285,270,290,280.0
4,HCME_A__20190513_01,A,PSP,2019-05-13 16:22,2019-05-15 17:35,9.1,0.796,69.65,3.54,19.6,50,0,768,137,672,120,260,235,280,257.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1543,HCME_A__20180828_01,A,Mars,2018-08-28 14:13,2018-09-03 19:19,31.9,1.383,-12.89,5.08,-10.9,-2,0,400,82,385,79,270,255,280,267.5
1544,HCME_A__20180919_01,A,Mars,2018-09-19 00:21,2018-09-23 08:01,4.3,1.382,-19.62,5.50,-19.6,0,4,633,26,554,23,275,260,290,275.0
1545,HCME_A__20181026_01,A,Mars,2018-10-26 04:32,2018-11-01 09:25,7.2,1.396,-33.68,5.55,-17.7,-16,-19,433,21,390,19,250,210,260,235.0
1546,HCME_A__20181211_01,A,Mars,2018-12-11 02:44,2018-12-17 09:33,0.4,1.437,-51.94,4.40,7.1,-59,8,402,1,396,1,275,260,280,270.0


In [8]:
ac_rec.id
ac_rec.target_name[5]

'SolarOrbiter'

In [9]:
ac_struct

array([(   0, 'HCME_A__20180828_01', 'A', 'PSP', 736934.59236111, 736938.83333333, 21.8, 0.969,  -7.92, 5.86,  -5.9,  -2.,   0., 400., 82., 396., 81., 270., 255., 280., 267.5),
       (   1, 'HCME_A__20180919_01', 'A', 'PSP', 736956.01458333, 736958.46944444,  2.4, 0.846, -13.51, 4.83, -13.5,   0.,   4., 633., 26., 597., 25., 275., 260., 290., 275. ),
       (   2, 'HCME_A__20181026_01', 'A', 'PSP', 736993.18888889, 736994.95208333,  2.1, 0.33 ,  10.47, 1.33,  26.5, -16., -19., 433., 21., 324., 16., 250., 210., 260., 235. ),
       ...,
       (1545, 'HCME_A__20181026_01', 'A', 'Mars', 736993.18888889, 736999.39236111,  7.2, 1.396, -33.68, 5.55, -17.7, -16., -19., 433., 21., 390., 19., 250., 210., 260., 235. ),
       (1546, 'HCME_A__20181211_01', 'A', 'Mars', 737039.11388889, 737045.39791667,  0.4, 1.437, -51.94, 4.4 ,   7.1, -59.,   8., 402.,  1., 396.,  1., 275., 260., 280., 270. ),
       (1547, 'HCME_A__20190126_01', 'A', 'Mars', 737085.04861111, 737091.29444444,  2.1, 1.493, -72.

In [10]:
ac_struct['id']
ac_struct['id'][100]

#comparison old and new method for arrival time prediction
deltata=(parse_time(ac.target_arrival_time[0:100]).plot_date-parse_time(ac_old.target_arrival_time).plot_date[0:100])*24
print(deltata)

[-1.15       -1.13333334  1.25       -5.6         0.96666667 -0.03333333
 -5.35       -2.75       -1.35       -0.78333333 -0.38333333  0.45
  2.18333333  0.48333333  4.46666667 -1.45        0.11666667 -4.28333333
 -2.21666667 -0.71666667 -1.58333333  0.9        -2.06666667  0.26666667
  7.76666666  0.18333333 -0.31666667  0.51666667  0.46666667  0.26666667
 -0.06666667  0.08333333 -0.33333333  0.         -0.1         0.1
 -0.28333333 -1.26666667  0.41666667  0.3        -0.71666667  0.06666667
 11.48333333  0.06666667 -0.21666667  0.28333333  5.          0.2
  2.38333333  0.28333333 -0.03333333  1.33333333  2.26666667  0.43333334
  4.43333333  0.          0.4        -0.23333333  0.08333333  0.58333333
 -0.15        0.16666667  0.28333333 -0.78333333 -0.13333333  0.28333333
 -0.15        0.08333333 -0.15        0.1         1.35        0.5
  4.03333333  0.43333334  0.08333333  0.03333333  0.1        -0.01666667
  1.86666667  0.1         0.65        0.13333333  0.03333333 -0.01666667
  0.9